<a href="https://colab.research.google.com/github/amos-fernandes/ufg-ext-LLM-Langchain-LlamaIndex/blob/main/2_Sentiment_Analysis_BoW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Sentimento

Análise de sentimento é o processo de automatizar a rotulação dos dados de texto de acordo com o sentimento da sentença (positivo, neutro e negativo). Análise de sentimento permite que as empresas possam analisar os dados em escala, detectando insights e automatizando processos.

Neste notebook, por exemplo, iremos analisar milhares de reviews do Play Store de aplicativos de compras. Ao invés de analisar estes dados manualmente, podemos usar análise de sentimento para entender automaticamente o que as pessoas estão pensando sobre os aplicativos, permitindo tomar decisões de negócio baseadas em dados.

<img src="https://drive.google.com/uc?id=1Qa0W_cwDOzb62xDUI8s0uleg0qJjWgN4" width="50%" height="50%">

## Análise de sentimento com BOW
Para construir um modelo de análise de sentimento utilizando a abordagem de vetorização BOW, precisamos de um conjunto de dados rotulados. Os algoritmos de aprendizado de máquina são bons com números e, por isso, temos que extrair ou converter os textos em números com a perda do mínimo de informações. Uma forma de fazer isso é utilizar o Bag-Of-Words (BOW) que atribui um número para cada palavra. Para vetorizar as palavras, podemos utilizar:

* **CountVectorizer**: conta o número de palavras no documento, ou seja, converte uma coleção de documentos de texto em uma matriz com a contagem de ocorrências de cada palavra no documento.
* **TfidfVectorizer**: TF-IDF (Term-Frequency-Inverse-Document Frequency) reduz o peso das palavras que ocorrem na maioria dos documentos e dá maior importância para palavras que aparecem em um subconjunto de documentos. Ou seja, o TF-IDF penaliza as palavras mais frequentes e dá maior importância para palavras raras em determinados documentos.

## Configurando o ambiente de execução

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth',1000)

## Explorando os dados de treinamento
Iremos baixar o arquivo `reviews_complete.csv` que contém quase **12k reviews** de aplicativos do PlayStore.

In [ ]:
!gdown 1wf-3sf_zkSi1VMhVSMcWfyX7wWVEY2R5

Downloading...
From: https://drive.google.com/uc?id=1wf-3sf_zkSi1VMhVSMcWfyX7wWVEY2R5
To: /content/reviews.csv
100% 6.78M/6.78M [00:00<00:00, 91.5MB/s]


In [ ]:
df = pd.read_csv("reviews.csv")

In [ ]:
df.shape

(12000, 13)

## Preparando o ambiente

Iremos utilizar a bilioteca nltk para facilitar o processamento dos textos. NLTK, que significa Natural Language Toolkit, é uma biblioteca disponível na linguagem Python com o intuito de realizar tarefas de NLP. É uma das principais bibliotecas de estudo para quem está ingressando nessa área, pois nela se encontram vários datasets e alguns algoritmos essenciais para análise e pré-processamento de textos.

Aqui utilizamos quatro pacotes do nltk:

* **stopwords**: pacote com uma lista de stopwords, incluíndo a língua portuguesa.
* **punkt**: pacote para tokenização que divide um texto em uma lista de sentenças utilizando um algoritmo não supervisionado.
* **wordnet**: banco de dados léxicos. Aqui utilizamos na operação de Lemmatization.
* **omw-1.4**: Open Multilingual Wordnet. Aqui utilizamos na operação de Lemmatization.

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
!pip install spacy
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
nlp = spacy.load('pt_core_news_sm')

## Pré-processamento dos dados

O pré-processamento dos dados textuais de treinamento envolvem as seguintes tarefas:

* **Normalização**: remoção de dígitos, pontuação, lower case, etc..
* **Tokenização**: segmenta o texto em palavras
* **Remoção de stopwords**: são um conjunto de palavras bastante utilizadas em uma linguagem e, geralmente, não tem importância para entender o significado principal do texto. Exemplo: "o", "da", "é" e "para".
* **Lemmatization**:  processo de deflexionar uma palavra para determinar o seu lema. Por exemplo, as palavras gato, gata, gatos, gatas são todas formas do mesmo lema: gato.

In [ ]:
doc = nlp(u'O tema do workshop até que é bom. O problema é o professor!')
doc.text.split()

['O',
 'tema',
 'do',
 'workshop',
 'até',
 'que',
 'é',
 'bom.',
 'O',
 'problema',
 'é',
 'o',
 'professor!']

In [ ]:
from textblob import Word
def pre_processing(df, stop_words):
    # Lower case
    df['content'] = df['content'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # remoção de dígitos ou números
    df['content'] = df['content'].str.replace('\d+', '')
    # Remoção de stop words
    df['content'] = df['content'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))
    # Lemmatization
    df['content'] = df['content'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
    return df
stop_words = stopwords.words('portuguese')
df = pre_processing(df, stop_words)

Pré-processamento dos dados utilizando `TfidfVectorizer` para construir o Bag of Word. Parâmetros:

* **ngram_range**: os limites inferiores e superiores da faixa de valores para os n-grams. Por exemplo, o ngram_range igual a (1,2) significa que podem ser gerados unigramas e bigramas.
* **sublinear_tf**: aplica um scaling no tf, ou seja, substitui o tf por 1 + log(tf).

In [ ]:
df['content'].head()

0                                                                                                                horrível . informação enganador , principalmente medida . dois produto mesmo suposto medida peito ombro , dois completamente diferente . claramente 2 3 tamanho acima outro . além caro qualidade apresentar material . primeiro último compra .
1    primeiro experiência plataformo . bom preço , variedade . entregar internacional prazo . entregar nacional péssima ! ! ! ! ainda recebir produto , atrasar com si nenhum contato . apenas rastreamento dizer q entregar .... qdo ? ? ? decepcionar . continuar nenhum informação pedido super atrasar ! ! ! . " suporte " responder nenhum contato . absurdo
2                     gostar nada de esse plataforma compra , fiz pedir devido problema sério endereço cancelar pedir , porém , cancelar endereço lá simplesmente existir tento contato suporte técnico consigo . horrível ! ! primeiro compra nunca mais , cliente shopee lá todo assistência preci

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
cv = TfidfVectorizer(ngram_range=(1,2),
                     sublinear_tf=True)
text_counts = cv.fit_transform(df['content'])

In [ ]:
text_counts[0]

<1x129590 sparse matrix of type '<class 'numpy.float64'>'
	with 51 stored elements in Compressed Sparse Row format>

Os dados devem ser divididos em conjunto de treinamento e de teste. O código abaixo aloca 75% da base de dados para treinamento e 25% para teste.

In [ ]:
#Dividindo os dados em treinamento e teste
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, df['sentiment'], test_size=0.25, random_state=5)

## Treinamento do modelo
O modelo será treinado utilizando os dados de treinamento `X_train` e `Y_train` utilizando Regressão Logística. Este modelo pode ser substituído por vários outros, tais como SVM, Random Forest e Decision Trees. Aqui a Regressão Logística é utilizada por ser computacionalmente eficiente no treinamento e predição.

In [ ]:
#Treinando o modelo
# from sklearn import svm
# SVM = svm.SVC(kernel='linear')
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(max_iter=300)
LR.fit(X_train, Y_train)

LogisticRegression(max_iter=300)

Na célula abaixo calculamos o score do modelo:

In [ ]:
from sklearn import metrics

predicted = LR.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted, Y_test)
print("Accuracy Score: ",accuracy_score)

Accuracy Score:  0.6426666666666667


## Predição com dados reais do Play Store

Abaixo utilizamos alguns textos extraídos de reviews do Play Store para verificar o quão o modelo está bom em textos reais. Utilizamos 3 sentenças (1 positiva, 1 neutra e 1 negativa) para realizarmos a avaliação do modelo em algumas sentenças escolhidas.

In [ ]:
class_names = ['negative', 'neutral', 'positive']

In [ ]:
negative_review = """Péssimo para quem quer vender. Além das altas taxas cobradas, demora para atualizar os encaminhamentos do produto"""
neutral_review = """Muito prático e fácil de usar, mas para conseguir ajuda é terrível!"""
positive_review = """Eu adoro o app enjoei, em todos os sentidos, acho bem intuitivo, fácil de procurar os ítens que você está querendo comprar e a interface é linda, bem clean e agradável aos olhos."""
reviews = [negative_review, neutral_review, positive_review]

In [ ]:
for review in reviews:
  review_vector = cv.transform([review]) # vetorização
  prediction = LR.predict(review_vector)[0]
  print(f'Review text: {review}')
  print(f'Sentiment  : {class_names[prediction]}')

Review text: Péssimo para quem quer vender. Além das altas taxas cobradas, demora para atualizar os encaminhamentos do produto
Sentiment  : negative
Review text: Muito prático e fácil de usar, mas para conseguir ajuda é terrível!
Sentiment  : positive
Review text: Eu adoro o app enjoei, em todos os sentidos, acho bem intuitivo, fácil de procurar os ítens que você está querendo comprar e a interface é linda, bem clean e agradável aos olhos.
Sentiment  : positive


# Referências

* https://www.kaggle.com/code/divsinha/sentiment-analysis-countvectorizer-tf-idf/notebook
* https://towardsdatascience.com/a-beginners-guide-to-sentiment-analysis-in-python-95e354ea84f6
* https://leportella.com/pt-br/npl-com-spacy/
* https://medium.com/@van3ssabandeira/o-famoso-spacy-90afb683b6fe
* https://www.alura.com.br/artigos/lemmatization-vs-stemming-quando-usar-cada-uma